In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import structural_similarity as ssim

from util import get_parking_spots_bboxes, empty_or_not

/home/ahmedcr7/.local/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.1.3 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
def get_diff(img1, img2):
    return np.abs(np.mean(img1)-np.mean(img2))

In [4]:
cap = cv2.VideoCapture('parking_1920_1080.mp4')

mask = cv2.imread('mask_1920_1080.png',0)
connected_components = cv2.connectedComponentsWithStats(mask, 8, cv2.CV_32S)

bbox = get_parking_spots_bboxes(connected_components)
counter = 0
predictions = [None for i in bbox]
diff = [None for i in bbox]

prev_frame = None
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if counter % 50 == 0:
        if prev_frame is None: # NumPy arrays can't be directly compared to None using the equality operator (==)
            for (idx,(x, y, w, h)) in enumerate(bbox):
                roi = frame[y:y + h, x:x + w]
                pred = empty_or_not(roi)
                predictions[idx] = pred
        
        else:
            diff = []
            for (idx,(x, y, w, h)) in enumerate(bbox):
                roi = frame[y:y + h, x:x + w]
                prev_roi = prev_frame[y:y + h, x:x + w]
                diff[idx] = get_diff(roi, prev_roi)
            
            indices_of_changed_roi = [j for j in np.argsort(diff) if diff[j]/np.amax(diff) > 0.4]

            for idx in indices_of_changed_roi:
                x,y,w,h = bbox[idx]
                roi = frame[y:y + h, x:x + w]
                pred = empty_or_not(roi)
                predictions[idx] = pred

    for (idx,(x, y, w, h)) in enumerate(bbox):
        if predictions[idx]:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        else:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

    counter += 1
    prev_frame = frame

cap.release()

# cv2.imshow('mask', mask)
# cv2.waitKey(0)
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ahmedcr7/.local/lib/python3.11/site-packages/cv2/qt/plugins"


IndexError: list assignment index out of range

: 

In [26]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

from util import get_parking_spots_bboxes, empty_or_not


def calc_diff(im1, im2):
    return np.abs(np.mean(im1) - np.mean(im2))


mask = 'mask_1920_1080.png'
video_path = 'parking_1920_1080.mp4'


mask = cv2.imread(mask, 0)

cap = cv2.VideoCapture(video_path)

connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S)

spots = get_parking_spots_bboxes(connected_components)

spots_status = [None for j in spots]
diffs = [None for j in spots]

previous_frame = None

frame_nmr = 0
ret = True
step = 30
while ret:
    ret, frame = cap.read()

    if not ret:
        break

    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_indx, spot in enumerate(spots):
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            diffs[spot_indx] = calc_diff(spot_crop, previous_frame[y1:y1 + h, x1:x1 + w, :])

        print([diffs[j] for j in np.argsort(diffs)][::-1])

    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]
        for spot_indx in arr_:
            spot = spots[spot_indx]
            x1, y1, w, h = spot

            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            spot_status = empty_or_not(spot_crop)

            spots_status[spot_indx] = spot_status

    if frame_nmr % step == 0:
        previous_frame = frame.copy()

    for spot_indx, spot in enumerate(spots):
        spot_status = spots_status[spot_indx]
        x1, y1, w, h = spots[spot_indx]

        if spot_status:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, 'Available spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))), (100, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_nmr += 1

cap.release()
cv2.destroyAllWindows()

[7.591036414565821, 7.32528180354268, 7.1615858737298055, 7.014159586873248, 6.439095928226351, 6.401851851851859, 6.110378634212296, 6.088122605363992, 6.0571895424836555, 5.906423331544119, 5.861111111111114, 5.743794769282033, 5.742295518907213, 5.624070317782298, 5.597826086956523, 5.5755772005772, 5.574183006535947, 5.572663139329805, 5.556302521008391, 5.540896218557393, 5.507366035292208, 5.419623521572547, 5.369315342328846, 5.301759834368532, 5.296618357487915, 5.263235294117649, 5.246798029556643, 5.236392914653791, 5.215059137098109, 5.198951994590942, 5.186162870945481, 5.165307971014485, 5.131535947712422, 5.116801893171058, 5.0926015085861, 5.053117380703583, 5.027260179434094, 5.019162640901769, 4.948275862068954, 4.9243156199678, 4.918242296918777, 4.901328273244786, 4.896881287726359, 4.754267310789061, 4.750700280112056, 4.7482492997198875, 4.736815415821496, 4.654303599374032, 4.6359477124183, 4.6250862663906105, 4.598200899550221, 4.5933123249299825, 4.5865905431122